In [1]:
import pandas as pd
import numpy as np
from resplitter.params import GroupNames, ResplitParams
from resplitter.resplit_builder import ResplitBuilder

In [2]:
df = pd.read_csv("TLO.csv",sep=";")

In [3]:
df['group_name'] = np.random.randint(0,3,len(df))
df['group_name'] = df['group_name'].replace(0,'control')
df['group_name'] = df['group_name'].replace(1,'target')
df['group_name'] = df['group_name'].replace(2,np.nan)

In [4]:
df.head()

,calc_id,test_name,category,moda_city,segment_name,segment_id,diff_day,test_post_start,test_post_end,customer_id,post_count_orders,post_revenue_orders,post_pg_orders,test_pre_start,test_pre_end,pre_count_orders,pre_revenue_orders,pre_gp_orders,group_name
0,349,OrdTLO_TU_WE_TH,test,Москва,"F - High freq, high spend",6,16,2022-02-16,2022-03-04,22496643,0,"0,00","0,00",2022-01-31,2022-02-15,1,"319,97",189.3538,control
1,349,OrdTLO_WD,test,Москва,"D - High freq, low spend",4,16,2022-02-16,2022-03-04,8731196,0,"0,00","0,00",2022-01-31,2022-02-15,2,"1438,83",966.3335,target
2,349,OrdTLO_HD,test,Москва,"D - High freq, low spend",4,16,2022-02-16,2022-03-04,14635792,3,"1759,89","857,6219",2022-01-31,2022-02-15,3,"1227,86",659.1377,control
3,349,OrdTLO,control,Москва,"D - High freq, low spend",4,16,2022-02-16,2022-03-04,8851051,3,"677,94","196,4858",2022-01-31,2022-02-15,2,"1299,92",635.9073,control
4,349,OrdTLO_TU_WE_TH,test,Москва,"D - High freq, low spend",4,16,2022-02-16,2022-03-04,13060936,3,"359,97","184,7897",2022-01-31,2022-02-15,3,"1248,91",613.6904,target


In [5]:
group_names = GroupNames(
    test_group_name = 'target',
    control_group_name = 'control'
)

resplit_params = ResplitParams(
    group_names = group_names,
    strata_col = 'test_name',
    group_col = 'group_name'
)

In [7]:
resplitter = ResplitBuilder(df, resplit_params)

In [9]:
resplited_df = resplitter.collect_new()

In [10]:
strata_all_count = df.value_counts('test_name').reset_index(name='all_count')

In [11]:
strata_test_counts = (df.value_counts(['test_name','group_name'])
                .reset_index(name = 'count')
)
strata_test_counts = strata_test_counts.merge(strata_all_count, on = 'test_name', how = 'inner')
strata_test_counts['frequency'] = strata_test_counts['count'] / strata_test_counts['all_count']

In [14]:
strata_test_counts

,test_name,group_name,count,all_count,frequency
0,OrdTLO,target,66707,200000,0.333535
1,OrdTLO,control,66700,200000,0.333500
2,OrdTLO_HD,target,33564,100000,0.335640
3,OrdTLO_HD,control,33073,100000,0.330730
4,OrdTLO_TU_WE_TH,control,33559,100000,0.335590
5,OrdTLO_TU_WE_TH,target,33102,100000,0.331020
6,OrdTLO_WD,target,23318,70000,0.333114
7,OrdTLO_WD,control,23297,70000,0.332814


In [15]:
strata_test_counts = (resplited_df.value_counts(['test_name','group_name'])
                .reset_index(name = 'count')
)
strata_test_counts = strata_test_counts.merge(strata_all_count, on = 'test_name', how = 'inner')
strata_test_counts['frequency'] = strata_test_counts['count'] / strata_test_counts['all_count']

In [16]:
strata_test_counts

,test_name,group_name,count,all_count,frequency
0,OrdTLO,control,66700,200000,0.333500
1,OrdTLO,target,66700,200000,0.333500
2,OrdTLO_TU_WE_TH,control,33102,100000,0.331020
3,OrdTLO_TU_WE_TH,target,33102,100000,0.331020
4,OrdTLO_HD,control,33073,100000,0.330730
5,OrdTLO_HD,target,33073,100000,0.330730
6,OrdTLO_WD,control,23297,70000,0.332814
7,OrdTLO_WD,target,23297,70000,0.332814
